In [2]:
import math
import random
from collections import deque
from dataclasses import dataclass
from typing import List, Tuple

import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


## 2a. Simplified Simulator

In [3]:
@dataclass
class AoIEnvConfig:
    num_nodes: int = 10          # number of graph nodes
    p: float = 0.8               # 1-hop propagation probability
    lambda_cost: float = 0.0     # action cost (can keep 0.0 for 2a)
    max_age: int = 100           # clip ages to avoid explosion
    max_steps: int = 50          # episode horizon
    init_age_low: int = 0        # initial ages range
    init_age_high: int = 10
    graph_type: str = "line"     # "line", "star", or "erdos"
    seed: int = SEED


class AoIEnv:
    """
    Simplified AoI environment with a Gym-like API:
      - reset() -> state
      - step(action) -> (next_state, reward, done, info)
      - render() for debugging.
    State:   age vector of shape (N,)
    Action:  integer in [0, N-1], index of node to refresh.
    """

    def __init__(self, config: AoIEnvConfig):
        self.cfg = config
        self.rng = np.random.RandomState(config.seed)
        self._build_graph()
        self.num_nodes = self.graph.number_of_nodes()
        self.state = None
        self.step_count = 0

        # Gym-style meta info
        self.observation_shape = (self.num_nodes,)
        self.action_n = self.num_nodes

    def _build_graph(self):
        if self.cfg.graph_type == "line":
            G = nx.path_graph(self.cfg.num_nodes)      # 0 - 1 - 2 - ... - N-1
        elif self.cfg.graph_type == "star":
            G = nx.star_graph(self.cfg.num_nodes - 1)
        elif self.cfg.graph_type == "erdos":
            while True:
                G = nx.erdos_renyi_graph(
                    self.cfg.num_nodes, 0.3, seed=self.cfg.seed
                )
                if nx.is_connected(G):
                    break
        else:
            raise ValueError(f"Unknown graph_type: {self.cfg.graph_type}")
        self.graph = G

    def seed(self, seed: int):
        self.cfg.seed = seed
        self.rng = np.random.RandomState(seed)

    def reset(self, init_ages: np.ndarray | None = None) -> np.ndarray:
        """Start a new episode. Returns initial age vector."""
        self.step_count = 0
        if init_ages is None:
            ages = self.rng.randint(
                self.cfg.init_age_low,
                self.cfg.init_age_high + 1,
                size=self.num_nodes,
            )
        else:
            ages = np.array(init_ages, dtype=np.float32)
            assert ages.shape == (self.num_nodes,)
        self.state = ages.astype(np.float32)
        return self.state.copy()

    def step(self, action: int) -> Tuple[np.ndarray, float, bool, dict]:
        """
        Apply one action:
          1) all ages +1
          2) chosen node resets to 0
          3) neighbors reset to 0 independently with prob p
        Returns: next_state, reward, done, info
        """
        assert 0 <= action < self.num_nodes, "invalid action index"

        ages = self.state.astype(np.float32)

        # 1) everyone ages by 1
        ages = ages + 1.0

        # 2) refresh chosen node
        ages[action] = 0.0

        # 3) 1-hop probabilistic propagation to neighbors
        for j in self.graph.neighbors(action):
            if self.rng.rand() < self.cfg.p:
                ages[j] = 0.0

        # clip ages
        ages = np.clip(ages, 0.0, float(self.cfg.max_age))

        self.state = ages
        self.step_count += 1

        # reward: negative average AoI minus lambda_cost
        avg_aoi = float(np.mean(self.state))
        reward = -avg_aoi - self.cfg.lambda_cost

        done = self.step_count >= self.cfg.max_steps
        info = {
            "avg_aoi": avg_aoi,
            "step": self.step_count,
            "action": action,
        }
        return self.state.copy(), reward, done, info

    def render(self):
        """Simple text render for debugging."""
        print(f"t={self.step_count} ages={self.state}")

In [9]:
cfg = AoIEnvConfig(num_nodes=6, graph_type="line", max_steps=5)
env = AoIEnv(cfg)

s = env.reset()
env.render()
for t in range(3):
    a = np.random.randint(env.action_n)
    ns, r, done, info = env.step(a)
    print(f"step {t}: action={a}, reward={r:.3f}, avg_aoi={info['avg_aoi']:.3f}")
    env.render()

t=0 ages=[ 6.  3. 10.  7.  4.  6.]
step 0: action=2, reward=-3.167, avg_aoi=3.167
t=1 ages=[7. 0. 0. 0. 5. 7.]
step 1: action=4, reward=-1.667, avg_aoi=1.667
t=2 ages=[8. 1. 1. 0. 0. 0.]
step 2: action=1, reward=-0.500, avg_aoi=0.500
t=3 ages=[0. 0. 0. 1. 1. 1.]


In [10]:
def random_policy(state: np.ndarray, env: AoIEnv) -> int:
    """Uniformly pick any node to refresh."""
    return env.rng.randint(env.action_n)


def greedy_stale_policy(state: np.ndarray, env: AoIEnv) -> int:
    """Pick the stalest node (max age)."""
    return int(np.argmax(state))


def degree_weighted_policy(state: np.ndarray, env: AoIEnv) -> int:
    """
    Pick node with largest age * degree.
    This tries to exploit propagation: high-degree nodes can affect more neighbors.
    """
    degrees = np.array([env.graph.degree[i] for i in range(env.num_nodes)], dtype=np.float32)
    scores = state * degrees
    return int(np.argmax(scores))


def rollout_policy(env: AoIEnv,
                   policy_fn,
                   num_episodes: int = 10,
                   horizon: int | None = None):
    """Generate trajectories for a given policy."""
    if horizon is None:
        horizon = env.cfg.max_steps

    trajectories = []

    for ep in range(num_episodes):
        s = env.reset()
        ep_data = {
            "states": [],
            "actions": [],
            "rewards": [],
            "next_states": [],
            "dones": [],
        }
        for t in range(horizon):
            a = policy_fn(s, env)
            ns, r, done, info = env.step(a)

            ep_data["states"].append(s.copy())
            ep_data["actions"].append(int(a))
            ep_data["rewards"].append(float(r))
            ep_data["next_states"].append(ns.copy())
            ep_data["dones"].append(bool(done))

            s = ns
            if done:
                break

        trajectories.append(ep_data)

    return trajectories


def save_flat_trajectories(trajectories, path: str):
    """
    Flatten episodes into a single table and save as compressed .npz.
    We also store episode_id and step_id to reconstruct episodes.
    """
    states = []
    actions = []
    rewards = []
    next_states = []
    dones = []
    episode_ids = []
    step_ids = []

    for ep_id, ep in enumerate(trajectories):
        for step_id, (s, a, r, ns, d) in enumerate(
            zip(ep["states"], ep["actions"], ep["rewards"],
                ep["next_states"], ep["dones"])
        ):
            states.append(s)
            actions.append(a)
            rewards.append(r)
            next_states.append(ns)
            dones.append(d)
            episode_ids.append(ep_id)
            step_ids.append(step_id)

    states = np.stack(states, axis=0)
    next_states = np.stack(next_states, axis=0)
    actions = np.array(actions, dtype=np.int64)
    rewards = np.array(rewards, dtype=np.float32)
    dones = np.array(dones, dtype=bool)
    episode_ids = np.array(episode_ids, dtype=np.int32)
    step_ids = np.array(step_ids, dtype=np.int32)

    np.savez_compressed(
        path,
        states=states,
        actions=actions,
        rewards=rewards,
        next_states=next_states,
        dones=dones,
        episode_ids=episode_ids,
        step_ids=step_ids,
    )
    print(f"Saved trajectories to {path}.npz with {states.shape[0]} steps.")

In [11]:
cfg = AoIEnvConfig(num_nodes=10, graph_type="line", max_steps=50, p=0.8)
env = AoIEnv(cfg)

# 1) Random policy trajectories
traj_random = rollout_policy(env, random_policy, num_episodes=20)
save_flat_trajectories(traj_random, "trajectories_random")

# 2) Greedy-stale trajectories
traj_greedy = rollout_policy(env, greedy_stale_policy, num_episodes=20)
save_flat_trajectories(traj_greedy, "trajectories_greedy")

# 3) degree-weighted
traj_deg = rollout_policy(env, degree_weighted_policy, num_episodes=20)
save_flat_trajectories(traj_deg, "trajectories_degree_weighted")

Saved trajectories to trajectories_random.npz with 1000 steps.
Saved trajectories to trajectories_greedy.npz with 1000 steps.
Saved trajectories to trajectories_degree_weighted.npz with 1000 steps.


In [12]:
import numpy as np

def load_npz_stats(path):
    data = np.load(path + ".npz")
    n = data["states"].shape[0]
    avg_return = float(data["rewards"].sum())
    avg_aoi = float(data["next_states"].mean())
    print(f"{path}: steps={n}, sum_reward={avg_return:.3f}, mean_next_state_AoI={avg_aoi:.3f}")
    return data

_ = load_npz_stats("trajectories_random")
_ = load_npz_stats("trajectories_greedy")
_ = load_npz_stats("trajectories_degree_weighted")

trajectories_random: steps=1000, sum_reward=-3231.700, mean_next_state_AoI=3.232
trajectories_greedy: steps=1000, sum_reward=-2120.400, mean_next_state_AoI=2.120
trajectories_degree_weighted: steps=1000, sum_reward=-2001.200, mean_next_state_AoI=2.001


In [16]:
def evaluate_policy(env, policy_fn, episodes=50):
    aoi_means = []
    for _ in range(episodes):
        s = env.reset()
        ep_aoi = []
        for t in range(env.cfg.max_steps):
            a = policy_fn(s, env)
            ns, r, done, info = env.step(a)
            ep_aoi.append(info["avg_aoi"])
            s = ns
            if done:
                break
        aoi_means.append(np.mean(ep_aoi))
    return float(np.mean(aoi_means)), float(np.std(aoi_means))

cfg_eval = AoIEnvConfig(num_nodes=10, graph_type="line", max_steps=50, p=0.8)
env_eval = AoIEnv(cfg_eval)

for name, pol in [
    ("random", random_policy),
    ("greedy_stale", greedy_stale_policy),
    ("degree_weighted", degree_weighted_policy),
]:
    m, sd = evaluate_policy(env_eval, pol, episodes=50)
    print(f"{name:16s} mean_AoI={m:.3f} ± {sd:.3f}")

random           mean_AoI=3.308 ± 0.371
greedy_stale     mean_AoI=2.107 ± 0.118
degree_weighted  mean_AoI=2.001 ± 0.123


## 2b. DQN Solution

In [29]:
env_line = AoIEnv(AoIEnvConfig(graph_type="line"))
env_star = AoIEnv(AoIEnvConfig(graph_type="star"))
env_erdos = AoIEnv(AoIEnvConfig(graph_type="erdos"))

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import numpy as np
from tqdm.notebook import tqdm

In [31]:
class QNet(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, action_dim)
        )
    def forward(self, x):
        return self.net(x)

class DQNAgent:
    def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.99, epsilon=1.0):
        self.q_net = QNet(state_dim, action_dim)
        self.target_net = QNet(state_dim, action_dim)
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.optim = optim.Adam(self.q_net.parameters(), lr=lr)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.995
        self.memory = deque(maxlen=10000)
        self.batch_size = 64
        self.steps = 0

    def act(self, state):
        if random.random() < self.epsilon:
            return random.randrange(self.q_net.net[-1].out_features)
        with torch.no_grad():
            q = self.q_net(torch.tensor(state, dtype=torch.float32).unsqueeze(0))
            return int(q.argmax().item())

    def push(self, s, a, r, ns, d):
        self.memory.append((s,a,r,ns,d))

    def train_step(self):
        if len(self.memory) < self.batch_size: return
        batch = random.sample(self.memory, self.batch_size)
        s,a,r,ns,d = zip(*batch)
        s = torch.tensor(s, dtype=torch.float32)
        a = torch.tensor(a).long()
        r = torch.tensor(r, dtype=torch.float32)
        ns = torch.tensor(ns, dtype=torch.float32)
        d = torch.tensor(d, dtype=torch.float32)

        q_vals = self.q_net(s).gather(1, a.unsqueeze(1)).squeeze()
        with torch.no_grad():
            target = r + self.gamma * (1-d) * self.target_net(ns).max(1)[0]
        loss = nn.MSELoss()(q_vals, target)
        self.optim.zero_grad(); loss.backward(); self.optim.step()

        if self.steps % 100 == 0:
            self.target_net.load_state_dict(self.q_net.state_dict())
        self.steps += 1
        self.epsilon = max(self.epsilon*self.epsilon_decay, self.epsilon_min)

In [32]:
def train_dqn(env, episodes=100, max_steps=50):
    agent = DQNAgent(env.num_nodes, env.num_nodes)
    for ep in tqdm(range(episodes)):
        s = env.reset()
        for t in range(max_steps):
            a = agent.act(s)
            ns, r, done, _ = env.step(a)
            agent.push(s,a,r,ns,done)
            agent.train_step()
            s = ns
            if done: break
    return agent

def evaluate_policy(env, policy_fn, episodes=30, max_steps=50):
    avg_aoi = []
    for _ in range(episodes):
        s = env.reset()
        ep_aoi = []
        for _ in range(max_steps):
            a = policy_fn(s)
            ns, r, done, info = env.step(a)
            ep_aoi.append(info["avg_aoi"])
            s = ns
            if done: break
        avg_aoi.append(np.mean(ep_aoi))
    return np.mean(avg_aoi), np.std(avg_aoi)

In [33]:
def random_policy(s): return np.random.randint(len(s))
def greedy_policy(s): return int(np.argmax(s))
def degree_weighted_policy_factory(G):
    deg = np.array([G.degree(i) for i in range(G.number_of_nodes())])
    def policy(s): return int(np.argmax(s * deg))
    return policy

In [39]:
graph_types = ["line", "star", "erdos"]
results = {}

for gtype in graph_types:
    env = AoIEnv(AoIEnvConfig(graph_type=gtype))
    print(f"\nTraining DQN on {gtype} graph...")
    dqn_agent = train_dqn(env, episodes=500)
    torch.save(dqn_agent.q_net.state_dict(), f"../content/artifacts/dqn_weights_{gtype}.pt")

    # policies
    deg_policy = degree_weighted_policy_factory(env.graph)
    pols = {
        "random": random_policy,
        "greedy": greedy_policy,
        "degree_weighted": deg_policy,
        "dqn": lambda s: int(torch.argmax(dqn_agent.q_net(torch.tensor(s, dtype=torch.float32))).item())
    }

    res = {}
    for name, p in pols.items():
        mean_aoi, std_aoi = evaluate_policy(env, p)
        res[name] = (mean_aoi, std_aoi)
        print(f"{gtype}-{name:15s}: mean_AoI={mean_aoi:.3f} ± {std_aoi:.3f}")
    results[gtype] = res


Training DQN on line graph...


  0%|          | 0/500 [00:00<?, ?it/s]

line-random         : mean_AoI=3.288 ± 0.354
line-greedy         : mean_AoI=2.116 ± 0.103
line-degree_weighted: mean_AoI=2.021 ± 0.126
line-dqn            : mean_AoI=2.053 ± 0.162

Training DQN on star graph...


  0%|          | 0/500 [00:00<?, ?it/s]

star-random         : mean_AoI=4.115 ± 0.914
star-greedy         : mean_AoI=3.597 ± 0.152
star-degree_weighted: mean_AoI=2.065 ± 0.307
star-dqn            : mean_AoI=0.280 ± 0.054

Training DQN on erdos graph...


  0%|          | 0/500 [00:00<?, ?it/s]

erdos-random         : mean_AoI=2.064 ± 0.257
erdos-greedy         : mean_AoI=1.355 ± 0.091
erdos-degree_weighted: mean_AoI=1.185 ± 0.112
erdos-dqn            : mean_AoI=1.216 ± 0.094


In [38]:
import pandas as pd
df = pd.DataFrame({g: {k: f"{v[0]:.3f}" for k,v in r.items()} for g,r in results.items()})
display(df)

,line,star,erdos
random,3.232,4.151,1.961
greedy,2.130,3.648,1.338
degree_weighted,2.047,1.949,1.200
dqn,2.223,0.245,1.308
